In [1]:
import numpy as np

In [2]:
v1 = np.array([-1, 2, -7, 9])
v2 = np.array([2, 8, 2, -1])

In [3]:
v1@v2

-9

In [4]:
A = np.matrix('1,1;1,2')
A

matrix([[1, 1],
        [1, 2]])

In [5]:
A@A.T

matrix([[2, 3],
        [3, 5]])

In [6]:
A = np.matrix('1,2;3,7')

In [7]:
np.linalg.inv(A)

matrix([[ 7., -2.],
        [-3.,  1.]])